# Проект телефонный справочник

Одним из выжных вопросов изучения истории позде имперского периода, 
является вопрос о соц мобильности. В историографии он является до сих пор нерешенным и вызывает споры.
Используемая официальная статистика дает разноречивые данные. 
Историки высказывают версии об улучшении благосостояния населения в конце 19 начале 20 века. Другие же приходят к прямопротивоположному выводу. 
Одним из подходов к решению этого вопроса может стать анализ справочника Весь Петербург охватывающий период с 1894-1917
В данном справочнике содержится информация, о жителях столицы империи,
их соц статусе, адресе проживания, и месте работы. Массив данных за 26 лет может позволить оценить темпы изменения социальной структуры Петербурга. 
Изменение указанных характеристик может позволить определить смену статуса и
материального состояния того, или иного человека. В частности: соц статус определенный табелем о рангах. Согласно установленному положению
смена рангов происходила по истечению трех лет службы, как по гражданской так и по военной специальности. 
Важно понять насколько это применялось на практике. Не меньшее значение имеет район проживания, который также был связан с соц статусом и 
материальным положением. Следует также обратить внимание на развитие телефонной сети. Обслуживание телефона было дорогостощим и лишь наиболее обеспеченные
люди могли себе его позволить. Именно эта информация и подверглась анализу в данном исследовании.

Данные были взяты из библиотеки, где справочник Весь Петербург оцифрован( https://vivaldi.nlr.ru/bx000011479/view#page=860. ) Далее данные прогонялись через распознаватель Adobe. Копируя оттуда, получали на выходе файл txt с достаточно "шумными" данными, который в последствии и подвергался обработки. Так выглядел исходник с которым работает код: https://github.com/ekatosha/ekatosha.github.io/blob/master/project/1.txt

Одной из первостепенных задач, встало приведение данных к минимальному единообразию. В правочнике информация представлена в виде Фамилия, Имя, Отчетство, статус, адрес, доп информация о должности, месте работы, телефоне, статусе домовладельца. Вся информация записана в столбик и достаточно большое количество строк может относится к одному человеку. Поэтому сначала, мы старались конконтинировать всю информацию связанную с одним человеком в одну строку. Для этого мы находили фамилии, это слова в начале строки, полностью прописные, и прикрепляли к ним новые строчки, до тех пор пока не встретим новое начало строки Полностью прописным, или не увидим символ - .Он означает нового человека, но с такой же фамилией. Для них дублировали значение фамилии, которую нашли выше. 

In [ ]:
import csv, re


with open('1.txt', 'r', encoding='utf-8') as f:
    content = f.read().split('\n')

people = []
year = content.pop(0)

human = ''
last_surname = ''
for s in content:
    try:
        first = s.split()[0]
    except IndexError:
        continue
    if first.isupper() and first[-1] != '.':
        last_surname = first
        people.append(human)
        human = s
    elif first[0] == '—':
        people.append(human)
        human = last_surname + s[1:]
    else:
        human += s

people.append(human)
people.pop(0)

Далее, в каждой строке искали имя, которое является вторым элементом строки, до этого проверяя не являтеся ли имя приставкой кн, гр, бар, что означает барон, князь, граф, и если нет, то добавляли это имя в список имен, собирая его до тех пор пока идет подряд не заглавные буквы. Это было нужно, тк текст очень насыщен шумом, и часто имя могло быть как Ив.(Иван) так и И в. И нужно, чтобы код собирал это воедино. Третий элемент списка относился к отчеству. Далее вся оставшаяся информация передается в поле Описание. И уже из этого поля мы искали сущности, соответствующие шаблону - адрес. Шаблону телефон. И также был составлен словарик, в котором хранятся статусы людей, по которым мы искали ключи сокращения, и выдавали значения, полное наименование статуса.

In [ ]:
statuses = {}
with open('dict2.csv', 'r', encoding='cp1251') as f:
    reader = csv.reader(f, delimiter=";")
    #reader = csv.reader(f.read()[1:].splitlines(), delimiter=";")
    for row in reader:
        statuses[row[0]] = row[1], row[2]

with open('result.csv', 'w', newline='', encoding='utf-8') as f:
    f.write(u'\ufeff')
    writer = csv.writer(f, delimiter=";")
    writer.writerow(['Год', 'Фамилия', 'Имя', 'Отчество', 'Статус', 'Адрес', 'Описание', 'Телефон'])
    for h in people:
        st = ""
        if len(h) < 3:
            continue
        h = h.split()
        sname = h.pop(0)
        if h[0] == "кн.":
            st = h.pop(0)
        if h[0] == "кп.":
            st = h.pop(0)
        if h[0] == "гр.":
            st = h.pop(0)
        if h[0] == "бар.":
            st = h.pop(0)
        fname = h.pop(0)
        try:
            while h[0][0].islower():
                fname += h.pop(0)
            mname = h.pop(0)
            #while h[0][0].islower():
                #mname += h.pop(0)
        
            description = ' '.join(h)
            description += " "
            description += st
        except IndexError:
            pass
        try:
            address = re.findall(r"\w+(?:\-?\s*?\w+?)?\.?\,?(?:\s+(?:пер|пл|наб|нер|кан|кап|пор)\.?)?,?\.?\s*\d+", description)[0]
        except IndexError:
            address = None
        try:
            tel = re.findall(r"(?:\Тлф|тлф)\.?\,?\—?\s*\d+", description)[0]
        except IndexError:
            tel = None

        matches = {}
        for key, value in statuses.items():
            try:
                matches[description.index(key)] = value[0]
            except ValueError:
                pass
        try:
            status = matches[min(matches.keys())]
        except ValueError:
            status = None
        writer.writerow([year, sname, fname, mname, status, address, description, tel])

На выходе получили файл формата csv. В котором в каждой колонке структурирована информация. результат https://github.com/ekatosha/ekatosha.github.io/blob/master/project/result.csv

Проект еще нуждается в доработке, нужно досоставить словарь, тк чем больше текста анализируется, тем больше находится именованных сущностей, которые еще не были включены. Также след этап проекта - сделать такой же словарь по профессиям. И Назначить индексы соц статуса, рангов, районов проживания для более удобного анализа данных. Самой главной проблемой источника является очень высокая зашумленность. Для первоначальной обработки были взяты уже сделанные сканы из библиотеки. Качество не предназначалось для распознавания и работы с объектов, а только для предварительного ознакомления, возможно если удастся расширить проект, то будут пересканированы данные из справочника, это устранит минимум половину шума. Весь уже имеющийся код можно скачать https://github.com/ekatosha/ekatosha.github.io/blob/master/project/project2.py .   

Несмтря на все недостатки, на основе полученных неполных данных уже можно сделать предварительные выводы. Справочник Весь Петербург далеко не во всех случаях содержит интересующую информацию 
о соц статусе человека.  Вместе с тем в тех случаях, когда содержится полная информация мы можем проследить соц мобильность с высокой степенью достоверности.
Следует отметить  в целом низкую горизонтальную и вертикальную мобильность населения, большая часть проанализированных людей показывает редкую смену 
места жительства и работы. В целом ряде случаев, повышение ранга соответсвует принятым нормам. Однако, есть ряд существенных исключений которые 
показывают нам, что некоторые гос ведомства являлись более быстрым социальным лифтом. В них прохождение по службе сокращается до одного-двух лет, вместо 
трех-четырех. Высокую соц мобильность, также показывает купеческое сословие. Это хорошо видно по смене места проживания из неблагополучных районов в престижные
в течении короткого времени. Можно также отметить значительный приток населения за счет миграции крестьян и других сословий и увеличения семей. 
Более детально проанализировать эту картину возможно только с привлечением данных их всего массива справочника. 
В частности по данным можно будет проследить практику заключения браков, что также является свидетельством изменения соц статуса. 